In [1]:
import pandas as pd
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [51]:
df = pd.read_csv('./association.csv')
df['DATE'] = df['DATE'].apply(lambda x: x.split(' ')[0])
df['Station number'] = df['Station number'].astype('object')
df['Station number'] = df['Station number'].apply(lambda x: str(x))

In [49]:
df.head()

,1 am_zscore,2 am_zscore,3 am_zscore,4 am_zscore,5 am_zscore,6 am_zscore,7 am_zscore,8 am_zscore,9 am_zscore,10 am_zscore,...,8 pm_zscore,9 pm_zscore,10 pm_zscore,11 pm_zscore,12 mid_zscore,TOTAL_zscore,DATE,Station number,Direction,COUNTY
0,z1,above extreme,above extreme,above extreme,above extreme,z1,-z1,-z2,-z2,-z2,...,-z1,-z1,z1,above extreme,z1,-z1,Jan,8,East,Norman
1,norm,-z1,-z1,z1,-z1,z1,z1,norm,norm,z1,...,z1,z1,z1,-z2,z2,z1,Jan,8,East,Norman
2,norm,-z1,z2,-z1,above extreme,z2,z1,above extreme,z2,norm,...,z2,z1,above extreme,-z1,-z1,above extreme,Jan,8,East,Norman
3,z2,norm,norm,-z1,-z1,z1,norm,above extreme,z2,above extreme,...,above extreme,above extreme,above extreme,above extreme,above extreme,above extreme,Jan,8,East,Norman
4,above extreme,above extreme,z2,z1,-z1,norm,above extreme,-z1,-z1,z1,...,z1,z2,norm,above extreme,z2,z1,Jan,8,East,Norman


In [50]:
def addNameToRow(r):
    for i in range(24):
        if('_' in r[i]):
            continue
        r[i] = r[i]+'_'+r.index[i].replace('_zscore','')
    return r
df.iloc[:,0:24] = df.iloc[:,0:24].apply(addNameToRow, axis=1)

KeyboardInterrupt: 

In [47]:
df.head(1)

,1 am_zscore,2 am_zscore,3 am_zscore,4 am_zscore,5 am_zscore,6 am_zscore,7 am_zscore,8 am_zscore,9 am_zscore,10 am_zscore,...,8 pm_zscore,9 pm_zscore,10 pm_zscore,11 pm_zscore,12 mid_zscore,TOTAL_zscore,DATE,Station number,Direction,COUNTY
0,z1_1 am,above extreme_2 am,above extreme_3 am,above extreme_4 am,above extreme_5 am,z1_6 am,-z1_7 am,-z2_8 am,-z2_9 am,-z2_10 am,...,-z1_8 pm,-z1_9 pm,z1_10 pm,above extreme_11 pm,z1_12 mid,-z1,Jan,8,East,Norman


In [53]:
dataset = df.values
oht = OnehotTransactions()
oht_ary = oht.fit(dataset).transform(dataset)
df_coded = pd.DataFrame(oht_ary, columns=oht.columns_)
df_coded.head()

KeyboardInterrupt: 

In [55]:
store = pd.HDFStore('store.h5')

store['df_coded'] = df_coded  # save it

NameError: name 'df_coded' is not defined

In [36]:
frequent_itemsets = apriori(df_coded, min_support=0.1, use_colnames=True)
frequent_itemsets.round(3).head()

,support,itemsets
0,0.919,[-z1]
1,0.604,[-z2]
2,0.195,[East]
3,0.144,[Hennepin]
4,0.305,[North]


In [37]:
len(frequent_itemsets)

171

In [39]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules.round(3)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(-z2),(-z1),0.604,0.919,0.592,0.981,1.068,0.038,4.340
1,(-z1),(-z2),0.919,0.604,0.592,0.645,1.068,0.038,1.116
2,(East),(-z1),0.195,0.919,0.181,0.927,1.009,0.002,1.110
3,(Hennepin),(-z1),0.144,0.919,0.131,0.910,0.990,-0.001,0.901
4,(North),(-z1),0.305,0.919,0.277,0.907,0.988,-0.003,0.881
5,(South),(-z1),0.305,0.919,0.281,0.922,1.004,0.001,1.045
6,(West),(-z1),0.195,0.919,0.180,0.922,1.004,0.001,1.047
7,(above extreme),(-z1),0.234,0.919,0.196,0.837,0.911,-0.019,0.499
8,(norm),(-z1),0.976,0.919,0.902,0.925,1.007,0.006,1.081
9,(-z1),(norm),0.919,0.976,0.902,0.982,1.007,0.006,1.366
